# Query database

Notebook for running Python sqlite3 commands for querying database.

Specific code blocks run separately to produce statistics.

## Setup:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
import sqlite3

In [ ]:
# path on rte
db_path = "/home/rte/data/db/arxiv_db_images.sqlite3"

In [ ]:
# path on kt
db_path = "/media/kt/scratch/rte/db/arxiv_db_images.sqlite3"

In [ ]:
# Here we import the sqlite3 database and create a cursor

db = sqlite3.connect(db_path)
c = db.cursor()

Get the pragma table info for each table

In [ ]:
c.execute('PRAGMA TABLE_INFO({})'.format("metadata"))
info = c.fetchall()

print("\nColumn Info:\nID, Name, Type, NotNull, DefaultVal, PrimaryKey")
for col in info:
    print(col)

In [ ]:
c.execute('PRAGMA TABLE_INFO({})'.format("images"))
info = c.fetchall()

print("\nColumn Info:\nID, Name, Type, NotNull, DefaultVal, PrimaryKey")
for col in info:
    print(col)

Quick test to retrive one row

In [ ]:
c.execute('''
    SELECT * 
    FROM images 
    ORDER BY ROWID ASC 
    LIMIT 1
''')
rows = c.fetchall()
for row in rows:
    print(row)

In [ ]:
c.execute('''
    SELECT * 
    FROM metadata 
    ORDER BY ROWID ASC 
    LIMIT 1
''')
rows = c.fetchall()
for row in rows:
    print(row)

## Writing to org mode

Find a few random entries from both metadata and images, write to org mode

In [ ]:
c.execute('''
    SELECT * 
    FROM metadata 
    ORDER BY RANDOM() 
    LIMIT 3
''')
rows = c.fetchall()
with open("metadata_sample.org", "w+") as write_file:
    print("|", file=write_file, end = '')
    for row in rows:
    #     print(row)
        for item in row:
            print(str(item).replace("\n", " "), file=write_file, end = '')
            print("|", file=write_file, end = '')
        print("\n|", file=write_file, end = '')
write_file.close()

In [ ]:
c.execute('''
    SELECT * 
    FROM images 
    ORDER BY RANDOM() 
    LIMIT 3
''')
rows = c.fetchall()
with open("images_sample.org", "w+") as write_file:
    print("|", file=write_file, end = '')
    for row in rows:
    #     print(row)
        for item in row:
            print(str(item).replace("\n", " "), file=write_file, end = '')
            print("|", file=write_file, end = '')
        print("\n|", file=write_file, end = '')
write_file.close()

## Database Queries

### Find any duplicate identifiers

In [ ]:
# look for duplicate rows in the metadata table
c.execute('''
    SELECT identifier, COUNT(identifier)
    FROM metadata
    GROUP BY identifier
    HAVING COUNT(identifier) > 1
    ''')
rows = c.fetchall()
for row in rows:
    print(row)

In [ ]:
# look for duplicate rows in the images table (brings up totals for images by article ID)
c.execute('''
    SELECT identifier, COUNT(identifier)
    FROM images
    GROUP BY identifier
    HAVING COUNT(identifier) > 0
    ''')
rows = c.fetchall()
for row in rows:
    print(row)

In [ ]:
# look for duplicate images (search by filename, filesize, and identifier)
# there is quite a lot of these, but that is how the source data was uploaded
c.execute('''
    SELECT identifier, filename, filesize, path, COUNT(*) c
    FROM images
    GROUP BY identifier, filename, filesize
    HAVING c > 1
    ''')
rows = c.fetchall()
for row in rows:
    print(row)

In [ ]:
print(len(rows))

### Earliest dates

In [ ]:
# Find the earliest date of an article

c.execute('''
    SELECT created, identifier
    FROM metadata
    WHERE created IS NOT NULL
    ORDER BY created ASC
    LIMIT 20
    ''')
rows = c.fetchall()
for row in rows:
    print(row)

In [ ]:
# Find the earliest dated articles and get the associated images

c.execute('''
    SELECT images.identifier, metadata.created, metadata.identifier 
    FROM images
    LEFT JOIN metadata ON images.identifier = metadata.identifier
    WHERE metadata.created IS NOT NULL
    ORDER BY created ASC
    LIMIT 40
    ''')
    
rows = c.fetchall()
for row in rows:
    print(row)

### Total numbers of articles and images

In [ ]:
# Find total number of rows in metadata

c.execute('''
    SELECT count(*)
    FROM metadata
    ''')
rows = c.fetchall()
for row in rows:
    print(row)

In [ ]:
# Find total number of rows in images

c.execute('''
    SELECT count(*)
    FROM images
    ''')
rows = c.fetchall()
for row in rows:
    print(row)

In [ ]:
# Find total number of rows in images (filtering out images without size or imageformat)

c.execute('''
    SELECT count(*)
    FROM images
    WHERE x IS NOT null AND x != ''
    AND y IS NOT null AND y != ''
    AND imageformat is not null AND imageformat != ''
    ''')
rows = c.fetchall()
for row in rows:
    print(row)
print(len(rows))    

### Average image sizes

In [ ]:
# Get average of the x size column

c.execute('''
    SELECT avg(x)
    FROM images
    ''')
rows = c.fetchall()
for row in rows:
    print(row)

In [ ]:
# Get average of the y size column

c.execute('''
    SELECT avg(y)
    FROM images
    ''')
rows = c.fetchall()
for row in rows:
    print(row)

In [ ]:
# Get median of the x size column

c.execute('''
    SELECT x
    FROM images
    ORDER by x
    LIMIT 1
    OFFSET (SELECT COUNT(*) FROM images) / 2
    ''')
rows = c.fetchall()
for row in rows:
    print(row)

In [ ]:
# Get median of the y size column

c.execute('''
    SELECT y
    FROM images
    ORDER by y
    LIMIT 1
    OFFSET (SELECT COUNT(*) FROM images) / 2
    ''')
rows = c.fetchall()
for row in rows:
    print(row)

### Missing fields

In [ ]:
# Find where there are NULL values for x

c.execute('''
    SELECT id, identifier, path, filename, filesize, x, y, imageformat
    FROM images
    WHERE x is null or x = ''
    OR y is null or y = ''
    OR imageformat is null or imageformat = ''
    ''')
rows = c.fetchall()
for row in rows:
    print(row)

In [ ]:
# generate a text file listing paths of files that couldn't be identified
with open("identify_errors_db.txt", "a+") as f:
    for row in rows[:]:
        filepath = row[2][1:] + "/" + row[3]
        print(str(row[0]) + "," + filepath)
        f.write(str(row[0]) + "," + filepath + "\n")


In [ ]:
print(len(rows))

In [ ]:
# Find where there are NULL values for y

c.execute('''
    SELECT id, identifier, path, filename, filesize, x, y, imageformat
    FROM images
    WHERE y is null or y = ''
    ''')
rows = c.fetchall()
for row in rows:
    print(row)

In [ ]:
print(len(rows))

In [ ]:
# Find where there are NULL values for identifier

c.execute('''
    SELECT id, identifier, path, filesize, x, y, imageformat
    FROM images
    WHERE identifier is null or identifier = ''
    ''')
rows = c.fetchall()
for row in rows:
    print(row)
    
# no result is a good result!

In [ ]:
# Find where there are NULL values for filesize

c.execute('''
    SELECT id, identifier, path, filesize, x, y, imageformat
    FROM images
    WHERE filesize is null or filesize = ''
    ''')
rows = c.fetchall()
for row in rows:
    print(row)

In [ ]:
print(len(rows))

In [ ]:
# Get number of articles with no category
# this seems to give no results

c.execute('''
    SELECT identifier, cat
    FROM metadata
    WHERE cat IS NULL OR cat = '' OR cat = 'None'
    ''')
rows = c.fetchall()
for row in rows:
    print(row)

In [ ]:
# Find where there is no matching metadata for a given image

c.execute('''
    SELECT count(images.identifier), images.identifier
    FROM images
    LEFT JOIN metadata ON metadata.identifier = images.identifier
    WHERE metadata.identifier IS NULL
    GROUP BY images.identifier
    ''')
rows = c.fetchall()
for row in rows:
    print(row)

In [ ]:
# Find where there is no identifier for an image

c.execute('''
    SELECT identifier, id, filename, filesize, path, x, y
    FROM images
    WHERE identifier IS NULL OR identifier = ''
    ''')
rows = c.fetchall()
for row in rows:
    print(row)

In [ ]:
# More looking for specific entries with no identifier

c.execute('''
    SELECT identifier, id
    FROM images
    WHERE identifier IS NULL OR identifier = ''
    ''')
rows = c.fetchall()
for row in rows:
    print(row)

In [ ]:
print(sum(row[0] for row in rows))

### Averages of images

In [ ]:
# Get total number of images per article

c.execute('''
    SELECT images.identifier, metadata.cat, count(images.identifier)    
    FROM images 
    LEFT JOIN metadata ON images.identifier = metadata.identifier 
    GROUP BY images.identifier
    ORDER BY count(images.identifier)
    ''')
rows = c.fetchall()
for row in rows:
    print(row)

In [ ]:
# Get total number of images per category

c.execute('''
    SELECT metadata.cat, count(images.identifier)
    FROM images
    LEFT JOIN metadata ON images.identifier = metadata.identifier 
    GROUP BY metadata.cat
    ORDER BY count(images.identifier) DESC    
    LIMIT 200
    ''')
rows = c.fetchall()
for row in rows:
    print(row)

In [ ]:
# Get total number of images per primary category only

c.execute('''
    SELECT substr(trim(metadata.cat),1,instr(trim(metadata.cat)||' ',' ')-1), count(images.identifier)
    FROM images
    LEFT JOIN metadata ON images.identifier = metadata.identifier 
    GROUP BY substr(trim(metadata.cat),1,instr(trim(metadata.cat)||' ',' ')-1)
    ORDER BY count(images.identifier) DESC    
    ''')
rows = c.fetchall()
for row in rows:
    print(row)

In [ ]:
# write text file

fname = "stats_images_by_category.txt"
f = open(fname, "w+")
for row in rows:
    f.write(row[0] + ", " + str(row[1]) + "\n")
f.close()

In [ ]:
print(len(rows))

In [ ]:
# Get total number of images per primary category only - with a specific filter

c.execute('''
    SELECT substr(trim(metadata.cat),1,instr(trim(metadata.cat)||' ',' ')-1), count(images.identifier)
    FROM images
    LEFT JOIN metadata ON images.identifier = metadata.identifier
    WHERE substr(trim(metadata.cat),1,instr(trim(metadata.cat)||' ',' ')-1) = "cond-mat.mes-hall"
    GROUP BY substr(trim(metadata.cat),1,instr(trim(metadata.cat)||' ',' ')-1)
    ORDER BY count(images.identifier) DESC    
    ''')
rows = c.fetchall()
for row in rows:
    print(row)

In [ ]:
print(len(rows))

### Totals by month/year

In [ ]:
# Find number of articles by year

c.execute('''
    SELECT count(metadata.identifier), strftime("%Y", metadata.created) as 'Y'
    FROM metadata
    GROUP BY strftime("%Y", metadata.created)
    ORDER BY strftime("%Y", metadata.created) ASC
    ''')
rows = c.fetchall()
for row in rows:
    print(row)

In [ ]:
# Find number of articles in each month/year

c.execute('''
    SELECT count(metadata.identifier), strftime("%m-%Y", metadata.created) as 'mY'
    FROM metadata
    GROUP BY strftime("%m-%Y", metadata.created)
    ORDER BY strftime("%Y", metadata.created) ASC
    ''')
rows = c.fetchall()
for row in rows:
    print(row)

In [ ]:
# Find number of images by year

c.execute('''
    SELECT count(images.identifier), strftime("%Y", metadata.created) as 'Y'
    FROM images
    LEFT JOIN metadata ON images.identifier = metadata.identifier 
    GROUP BY strftime("%Y", metadata.created)
    ORDER BY strftime("%Y", metadata.created) ASC
    ''')
rows = c.fetchall()
for row in rows:
    print(row)

In [ ]:
# Find number of images in each month/year

c.execute('''
    SELECT count(images.identifier), strftime("%m-%Y", metadata.created) as 'mY'
    FROM images
    LEFT JOIN metadata ON images.identifier = metadata.identifier 
    GROUP BY strftime("%m-%Y", metadata.created)
    ORDER BY strftime("%Y", metadata.created) ASC
    ''')
rows = c.fetchall()
for row in rows:
    print(row)

In [ ]:
total = 0
for row in rows:
    total += row[0]
print(total)

#### ...and by category

In [ ]:
# Find number of articles by year by category

c.execute('''
    SELECT count(metadata.identifier), strftime("%Y", metadata.created) as 'Y'
    FROM metadata
    WHERE substr(trim(cat),1,instr(trim(cat)||' ',' ')-1) = 'cs.CV'
    GROUP BY strftime("%Y", metadata.created)
    ORDER BY strftime("%Y", metadata.created) ASC
    ''')
rows = c.fetchall()
for row in rows:
    print(row)

In [ ]:
# Find number of articles by year by category, for a specific query

c.execute('''
    SELECT count(metadata.identifier), strftime("%Y", metadata.created) as 'Y'
    FROM metadata
    WHERE substr(trim(cat),1,instr(trim(cat)||' ',' ')-1) = 'cs.CV'
    AND strftime("%Y", metadata.created) = '2018'
    GROUP BY strftime("%Y", metadata.created)
    ORDER BY strftime("%Y", metadata.created) ASC
    ''')
rows = c.fetchall()
for row in rows:
    print(row)

In [ ]:
# Find number of articles by year by category, for a specific query

c.execute('''
    SELECT count(metadata.identifier), strftime("%Y", metadata.created) as 'Y'
    FROM metadata
    WHERE substr(trim(cat),1,instr(trim(cat)||' ',' ')-1) = 'cs.CV'
    AND strftime("%Y", metadata.created) = '2012'
    GROUP BY strftime("%Y", metadata.created)
    ORDER BY strftime("%Y", metadata.created) ASC
    ''')
rows = c.fetchall()
for row in rows:
    print(row)

In [ ]:
print(len(rows))

In [ ]:
# values = np.array()
values = [item[0] for item in rows]
plt.plot(values)

### Other metadata

In [ ]:
# Find number of images per each different imageformat

c.execute('''
    SELECT imageformat, count(imageformat)
    FROM images
    GROUP BY imageformat
    ORDER BY count(imageformat) DESC
    ''')
rows = c.fetchall()
for row in rows:
    print(row)

In [ ]:
# Get only the first listed category

c.execute('''
    SELECT identifier, cat, substr(trim(cat),1,instr(trim(cat)||' ',' ')-1)
    FROM metadata
    LIMIT 20
    ''')
rows = c.fetchall()
for row in rows:
    print(row)

In [ ]:
# Get the licence information for number of articles

c.execute('''
    SELECT licence, COUNT(licence)
    FROM metadata
    GROUP BY licence
    HAVING COUNT(identifier) > 0
    ''')
rows = c.fetchall()
for row in rows:
    print(row)

In [ ]:
# Get the number of images per licence

c.execute('''
    SELECT metadata.licence, COUNT(metadata.licence)
    FROM images
    LEFT JOIN metadata ON images.identifier = metadata.identifier
    GROUP BY metadata.licence
    HAVING COUNT(images.identifier) > 0
    ''')
rows = c.fetchall()
for row in rows:
    print(row)

In [ ]:
# Get number of articles for each primary category

c.execute('''
    SELECT count(substr(trim(cat),1,instr(trim(cat)||' ',' ')-1)), substr(trim(cat),1,instr(trim(cat)||' ',' ')-1)
    FROM metadata
    GROUP BY substr(trim(cat),1,instr(trim(cat)||' ',' ')-1)
    ''')
rows = c.fetchall()
for row in rows:
    print(row)

### Querying by image extension

In [ ]:
# define reverse function
db.create_function("reverse", 1, lambda s: s[::-1])

In [ ]:
# Get the number of images per extension
# DON'T filter based on X or Y dimension

c.execute('''
    SELECT COUNT(reverse(substr(reverse(filename),1,instr(reverse(filename),'.')-1))), reverse(substr(reverse(filename),1,instr(reverse(filename),'.')-1)) AS extension
    FROM images
    GROUP BY extension 
    ''')

rows = c.fetchall()
for row in rows:
    print(row)

In [ ]:
# Get the number of images per extension
# filter based on X or Y dimension

c.execute('''
    SELECT COUNT(reverse(substr(reverse(filename),1,instr(reverse(filename),'.')-1))), reverse(substr(reverse(filename),1,instr(reverse(filename),'.')-1)) AS extension
    FROM images
    WHERE x is not null and x != ''
    AND y is not null and y != ''
    AND imageformat is not null and imageformat != ''
    GROUP BY extension
    ''')

rows = c.fetchall()
for row in rows:
    print(row)

In [ ]:
# Get the number of images per extension by year
# filter based on X or Y dimension
# 2018 only

c.execute('''
    SELECT reverse(substr(reverse(filename),1,instr(reverse(filename),'.')-1)) AS extension, COUNT(reverse(substr(reverse(filename),1,instr(reverse(filename),'.')-1)))
    FROM images
    LEFT JOIN metadata ON images.identifier = metadata.identifier
    WHERE x is not null and x != ''
    AND y is not null and y != ''
    AND imageformat is not null and imageformat != ''
    AND strftime("%Y", metadata.created) = '2018'
    GROUP BY extension
    ''')

rows = c.fetchall()
for row in rows:
    print(row)

In [ ]:
# Get the number of images per extension
# filter based on X or Y dimension

c.execute('''
    SELECT reverse(substr(reverse(filename),1,instr(reverse(filename),'.')-1)) AS extension, COUNT(reverse(substr(reverse(filename),1,instr(reverse(filename),'.')-1)))
    FROM images
    LEFT JOIN metadata ON images.identifier = metadata.identifier
    WHERE x is not null and x != ''
    AND y is not null and y != ''
    AND imageformat is not null and imageformat != ''
    GROUP BY extension
    ''')

rows = c.fetchall()
for row in rows:
    print(row)
    
total = 0
for row in rows:
    total += row[1]
print("total:", total)

print("*" * 20)

alldata = []
for i, row in enumerate(rows):
    alldata.append([row[0], row[1], row[1]/total])
for d in alldata:
    print("{} | {} | {:2.2%}".format(d[0], d[1], d[2]))

### Get extensions by years and get statistics for changing percentages

In [ ]:
# Get list of years

c.execute('''
    SELECT strftime("%Y", metadata.created), COUNT(strftime("%Y", metadata.created))
    FROM metadata
    GROUP BY strftime("%Y", metadata.created)
    ''')

years = []

rows = c.fetchall()
for row in rows:
    print(row)
    years.append(row[0])
print(years)

In [ ]:
# Get the number of images per extension by year
# filter based on X or Y dimension

data = []

# manually selecting years
# years = ["2000", "2005", "2010", "2015"]

db.create_function("reverse", 1, lambda s: s[::-1])
sql = ('''
    SELECT reverse(substr(reverse(filename),1,instr(reverse(filename),'.')-1)) AS extension, COUNT(reverse(substr(reverse(filename),1,instr(reverse(filename),'.')-1)))
    FROM images
    LEFT JOIN metadata ON images.identifier = metadata.identifier
    WHERE x is not null and x != ''
    AND y is not null and y != ''
    AND imageformat is not null and imageformat != ''
    AND strftime("%Y", metadata.created) = ?
    GROUP BY extension
    ''')

# data = []

for y, year in enumerate(years):
    print("*" * 20)
    print(year)
    print("*" * 20)
    c.execute(sql, (year, ))

    rows = c.fetchall()
#     for row in rows:
#         print(row)
    total = 0
    for row in rows:
        total += row[1]
    print("total:", total)

    data = []
    for i, row in enumerate(rows):
        data.append([row[0], row[1], row[1]/total])
    for d in data:
        print("{} | {} | {:2.2%}".format(d[0], d[1], d[2]))
#     print(data)
        

In [ ]:
for d in data:
    print(d)

In [ ]:
# get proportion as percentage

total = 0
for row in rows:
    total += row[1]
print(total)

data = []
for row in rows:
    data.append([row[0], row[1], row[1]/total])
print(data)

In [ ]:
# take second element for sort
def takeSecond(elem):
    return elem[1]
data.sort(key=takeSecond)

for d in data:
    print("{} | {} | {:2.2%}".format(d[0], d[1], d[2]))

In [ ]:
values = []
names = []
for row in rows:
    values.append(row[0])
    names.append(row[1])

In [ ]:
values_save = values
names_save = names

In [ ]:
for i, name in enumerate(names.copy()):
    names[i] = name.lower()
print(names)

In [ ]:
print(values)
print(names)
print(len(values))
print(len(names))

In [ ]:
total = 0
for row in rows:
    total += row[0]
print(total)

In [ ]:
# writing file to org mode for github

# write the data to a file
with open("stats_articles_by_cat.org", "w") as write_file:
    print("* number of articles by category", file=write_file)
    print("|-|-|", file=write_file)
    for cat in rows:
#         joined = list(zip(cat[1], cat[2]))
        #     print(joined)
        print('|' + str(cat[0]) + "|" + str(cat[1]) + "|", file=write_file)
    print("|-|-|", file=write_file)
write_file.close()

#### Image metadata

In [ ]:
# Get the number of images per licence

c.execute('''
    SELECT images.creator, COUNT(images.creator)
    FROM images
    GROUP BY images.creator
    ORDER BY COUNT(images.creator) DESC
    ''')
rows = c.fetchall()
for row in rows:
    print(row)

In [ ]:
print("|-|-|-|")
for row in rows[1:16]:
    per = str(round((row[1]/10053059*100), 2))
    print("|" + per + "|" + str(row[1]) + "|" + row[0] + '|')
print("|-|-|-|")